[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# Covid-19 Dashboard

This dashboard displays Covid-19 data in Wales, including the number of tests and cases

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

#global variable for dataframe
global df
df={}

## Load initial data from disk

This loads the initial data

In [3]:
# Load initial data from json file
with open("timeaxis.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

## Wrangle the data

The section below adds the key data into a dataframe

In [4]:
#convert date strings into pandas datetime object

def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

#create dataframe with dates as index and fills with 'tests' and 'cases' as column values
def wrangle_data(rawdata):

    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
          
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    timeaxisdf=pd.DataFrame(index=index, columns=['tests', 'cases'])
    for entry in datalist: # each entry is a dictionary with date, tests and cases
        date=parse_date(entry['date'])
        for column in ['tests', 'cases']:
            if pd.isna(timeaxisdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                timeaxisdf.loc[date, column]=value
    timeaxisdf.fillna(0.0, inplace=True)
    return timeaxisdf
df=wrangle_data(jsondata) # df is the dataframe for plotting
df

,tests,cases
2020-03-04,0.0,0.1
2020-03-05,0.0,0.0
2020-03-06,0.0,0.0
2020-03-07,0.0,0.1
2020-03-08,0.0,0.1
...,...,...
2023-11-28,0.0,3.8
2023-11-29,0.0,4.3
2023-11-30,0.0,4.0
2023-12-01,0.0,4.1


## Download current data

The code below allows refreshing the data, and refreshes the graph accordingly

In [5]:
def access_api():
    filters = [
        'areaType=nation',
        'areaName=Wales'
    ]

    structure = {
        "date": "date",
        "cases": "newCasesBySpecimenDateRollingRate",
        "beds": "covidOccupiedMVBeds",
        "hospital": "newAdmissionsRollingRate",
        "deaths": "newDailyNsoDeathsByDeathDate",
        "tests": "newPCRTestsByPublishDateRollingSum",
    }

    apidata = Cov19API(filters=filters, structure=structure)

    apidata=apidata.get_json()
    return apidata # return data read from the API

In [6]:
#Define operations once button is clicked: new set of API data retrieved and new dataframe is created from this
def api_button_callback(button):
    apidata=access_api()
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
    apibutton.disabled=True
    apibutton.description = "Data Fetched"

    
apibutton=wdg.Button(
    description='Fetch Data',
    disabled=False,
    button_style='info',
    tooltip="Click to fetch current Public Health England data",
    icon='download'
)

#Define calling of callback function on click of button
apibutton.on_click(api_button_callback)

display(apibutton)

Button(button_style='info', description='Fetch Data', icon='download', style=ButtonStyle(), tooltip='Click to …

## Graphs and Analysis

The code below creates a graph with the SelectMultiple and RadioButtons widgets

In [7]:
def timeaxis_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

series=wdg.SelectMultiple(
    options=['tests', 'cases'],
    value=['tests', 'cases'],
    rows=2,
    description='Stats:',
    disabled=False
)
#adding log to linear scale toggle button
scale=wdg.RadioButtons(
    options=['linear', 'log'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)
# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def refresh_graph():
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other 
    scale.value=current 

graph=wdg.interactive_output(timeaxis_graph, {'list(gcols)': series})



# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(timeaxis_graph, {'gcols': series ,'gscale': scale})

display(controls, graph)

Output()

The graph above shows both the number of tests and cases over time. The Stats menu can be altered to show just tests or just cases or both.

The Scale function allows switching between a linear and log scale.